In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf
import keras
import matplotlib.pyplot as plt
import cv2 
import os
import pickle
!pip install imutils
import imutils
import pathlib
import math
import gc

from tqdm import tqdm
import keras_tuner as kt
import tensorflow.keras.backend as K
from sklearn.model_selection import StratifiedKFold

DATASET_PATH = r"../input/neoplasie/T1VOL//"

In [ ]:
def deskull(path, IMG_SIZE, direct_mode = False):
    if not direct_mode:
        img = cv2.imread(path)
    else:
        img = cv2.cvtColor(path, cv2.COLOR_BGR2RGB)

    img = cv2.resize(
                img,
                dsize=IMG_SIZE,
                interpolation=cv2.INTER_CUBIC
        )
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    gray = cv2.GaussianBlur(gray, (5, 5), 0)

    # threshold the image, then perform a series of erosions +
    # dilations to remove any small regions of noise
    thresh = cv2.threshold(gray, 45, 255, cv2.THRESH_BINARY)[1]
    thresh = cv2.erode(thresh, None, iterations=2)
    thresh = cv2.dilate(thresh, None, iterations=2)

    # find contours in thresholded image, then grab the largest one
    cnts = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cnts = imutils.grab_contours(cnts)
    c = max(cnts, key=cv2.contourArea)

    # find the extreme points
    extLeft = tuple(c[c[:, :, 0].argmin()][0])
    extRight = tuple(c[c[:, :, 0].argmax()][0])
    extTop = tuple(c[c[:, :, 1].argmin()][0])
    extBot = tuple(c[c[:, :, 1].argmax()][0])

    # add contour on the image
    img_cnt = cv2.drawContours(img.copy(), [c], -1, (0, 255, 255), 4)

    # add extreme points
    img_pnt = cv2.circle(img_cnt.copy(), extLeft, 8, (0, 0, 255), -1)
    img_pnt = cv2.circle(img_pnt, extRight, 8, (0, 255, 0), -1)
    img_pnt = cv2.circle(img_pnt, extTop, 8, (255, 0, 0), -1)
    img_pnt = cv2.circle(img_pnt, extBot, 8, (255, 255, 0), -1)

    # crop
    ADD_PIXELS = 0
    img2 = img[extTop[1]-ADD_PIXELS:extBot[1]+ADD_PIXELS, extLeft[0]-ADD_PIXELS:extRight[0]+ADD_PIXELS].copy()
    img_final = cv2.resize(
        img2,
        dsize=IMG_SIZE,
        interpolation=cv2.INTER_CUBIC
    )
    return img_final.copy()

In [ ]:
def read_dataset(dataframe, indexes=[]):
    X_train = []
    Y_train = []

    if len(indexes)>0:
        source = indexes
    else:
        source = dataframe.index
    
    for index in source:
        row = dataframe.iloc[index]
        current_path = pathlib.PureWindowsPath(row["Path"]).as_posix()
        X_train.append(DATASET_PATH + current_path)
        #X_train.append(deskull(DATASET_PATH + current_path,(512,512))/255)
        if row["Class"] == 'MET':
            Y_train.append(1)
        else:
            Y_train.append(0)
            
    return np.array(X_train), np.array(Y_train)

In [ ]:
dataframe = pd.read_csv(DATASET_PATH + 'dataset.csv')
folds = pickle.load(open(DATASET_PATH + 'folds.npy', "rb"))

In [ ]:
#SHUFFLE
for i in range(0,len(folds)):
    indices = np.arange(folds[i].shape[0])
    np.random.shuffle(indices)
    folds[i] = folds[i][indices]

In [ ]:
print(folds[9])

In [ ]:
X_train, Y_train = read_dataset(dataframe, [*folds[8], *folds[9], *folds[2], *folds[3], *folds[4], *folds[5], *folds[6], *folds[7]])
X_val, Y_val = read_dataset(dataframe, [*folds[0],*folds[1]])

In [ ]:
class VOLSequence(tf.keras.utils.Sequence):

    def __init__(self, x_set, y_set, batch_size, deskull=True, transform=None):
        self.x, self.y = x_set, y_set
        self.batch_size = batch_size
        self.transform = transform
        self.deskull = deskull

    def __len__(self):
        return math.ceil(len(self.x) / self.batch_size)
    
    def read_img(self, path):
        img = cv2.imread(path)
        img = cv2.resize(img, dsize=(512, 512), interpolation=cv2.INTER_CUBIC)
        return img.copy()
                         
    def __getitem__(self, idx):
        batch_x = self.x[idx * self.batch_size:(idx + 1) *
        self.batch_size]
        batch_y = self.y[idx * self.batch_size:(idx + 1) *
        self.batch_size]
        
        if self.transform is None:
            if self.deskull:
                return np.array([
                    deskull(file_name, (512, 512))
                       for file_name in batch_x])/255, np.array(batch_y)
            else:
                return np.array([
                    self.read_img(file_name)
                       for file_name in batch_x])/255, np.array(batch_y)
        else:
            if self.deskull:
                return np.array([
                    deskull(self.transform(cv2.imread(file_name)), (512, 512), True)
                       for file_name in batch_x])/255, np.array(batch_y)
            else:
                return np.array([
                    self.transform(self.read_img(file_name))
                       for file_name in batch_x])/255, np.array(batch_y)

In [ ]:
Train_set = VOLSequence(X_train, Y_train, deskull=True, batch_size=32)
Val_set = VOLSequence(X_val, Y_val, deskull=True, batch_size=32)

# Pre-processing (Crop)

In [ ]:
#Esempio su una immagine
IMG_SIZE = (512,512)
img = cv2.imread('../input/neoplasie/T1VOL/volGBM/VOL0001_GBM_IDH1MUT.jpg')
img = cv2.resize(
            img,
            dsize=IMG_SIZE,
            interpolation=cv2.INTER_CUBIC
        )
gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
gray = cv2.GaussianBlur(gray, (5, 5), 0)

# threshold the image, then perform a series of erosions +
# dilations to remove any small regions of noise
thresh = cv2.threshold(gray, 45, 255, cv2.THRESH_BINARY)[1]
thresh = cv2.erode(thresh, None, iterations=2)
thresh = cv2.dilate(thresh, None, iterations=2)

# find contours in thresholded image, then grab the largest one
cnts = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
cnts = imutils.grab_contours(cnts)
c = max(cnts, key=cv2.contourArea)

# find the extreme points
extLeft = tuple(c[c[:, :, 0].argmin()][0])
extRight = tuple(c[c[:, :, 0].argmax()][0])
extTop = tuple(c[c[:, :, 1].argmin()][0])
extBot = tuple(c[c[:, :, 1].argmax()][0])

# add contour on the image
img_cnt = cv2.drawContours(img.copy(), [c], -1, (0, 255, 255), 4)

# add extreme points
img_pnt = cv2.circle(img_cnt.copy(), extLeft, 8, (0, 0, 255), -1)
img_pnt = cv2.circle(img_pnt, extRight, 8, (0, 255, 0), -1)
img_pnt = cv2.circle(img_pnt, extTop, 8, (255, 0, 0), -1)
img_pnt = cv2.circle(img_pnt, extBot, 8, (255, 255, 0), -1)

# crop
ADD_PIXELS = 0
new_img = img[extTop[1]-ADD_PIXELS:extBot[1]+ADD_PIXELS, extLeft[0]-ADD_PIXELS:extRight[0]+ADD_PIXELS].copy()


In [ ]:
plt.figure(figsize=(15,6))
plt.subplot(141)
plt.imshow(img)
plt.xticks([])
plt.yticks([])
plt.title('Step 1. Get the original image')
plt.subplot(142)
plt.imshow(img_cnt)
plt.xticks([])
plt.yticks([])
plt.title('Step 2. Find the biggest contour')
plt.subplot(143)
plt.imshow(img_pnt)
plt.xticks([])
plt.yticks([])
plt.title('Step 3. Find the extreme points')
plt.subplot(144)
plt.imshow(new_img)
plt.xticks([])
plt.yticks([])
plt.title('Step 4. Crop the image')
plt.show()

# Architettura Rete

In [ ]:
# VGG16
from keras.applications.vgg16 import VGG16, preprocess_input
from keras.models import Model
from keras.layers import Dense
from keras.layers import Flatten, GlobalAveragePooling2D, Dropout

def get_model():
    # load model without classifier layers
    base_model = VGG16(include_top=False, input_shape=(512, 512, 3))
    base_model.trainable = False

    inputs = keras.Input(shape=(512, 512, 3))
    #x = preprocess_input(inputs)
    base_model = base_model(inputs, training=False)

    # add new classifier layers
    flat1 = Flatten()(base_model)
    out = Dropout(0.5)(flat1)
    output = Dense(1, activation='sigmoid')(out)
    # define new model
    model = Model(inputs=[inputs], outputs=[output])
    return model

#Metriche
def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [ ]:
model = get_model()
keras.utils.plot_model(model)

# Ricerca iperparametri

In [ ]:
def build_model(hp):
    # load model without classifier layers
    base_model = VGG16(include_top=False, input_shape=(512, 512, 3))
    base_model.trainable = False

    inputs = keras.Input(shape=(512, 512, 3))
    x = preprocess_input(inputs)
    base_model = base_model(x, training=False)

    flat1 = Flatten()(base_model)
    out = Dropout(0.5)(flat1)
    output = Dense(1, activation='sigmoid')(out)
    model = Model(inputs=[inputs], outputs=[output])
    
    lr = hp.Float(
        'learning_rate',
        min_value=1e-5,
        max_value=1e-2,
        sampling='LOG',
        default=1e-3
    )
    #keras.optimizers.RMSprop(lr=1e-4)
    model.compile(optimizer=keras.optimizers.Adam(lr), loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),metrics=keras.metrics.BinaryAccuracy())
    return model

In [ ]:
tuner = kt.RandomSearch(
    build_model,
    objective='val_loss',
    max_trials=5)

tuner.search(training_set, epochs=5, validation_data=valid_set)
best_model = tuner.get_best_models()[0]

In [ ]:
tuner.results_summary()
model = best_model

# Addestramento

In [ ]:
callbacks = []
runPath = './models/'
callbacks.append(keras.callbacks.ModelCheckpoint(runPath + '/weights.{epoch:02d}-{val_loss:.2f}.hdf5', monitor='val_loss', verbose=1, save_best_only=True, save_weights_only = False, mode='min', save_freq='epoch'))


In [ ]:
model = get_model()
model.compile(optimizer=keras.optimizers.Adam(lr=0.00005), loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),metrics=['binary_accuracy',f1_m,precision_m, recall_m])
history = model.fit(Train_set, epochs=25, validation_data=Val_set, callbacks=callbacks)

In [ ]:
keras.utils.plot_model(model)

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sn

labels = ["GBM (0)","MET (1)"]

#Y_test = np.argmax(y_test, axis=1) # Convert one-hot to index
Y_test = Val_set.y
y_pred = model.predict(Val_set)
rounded_pred = np.round(abs(y_pred))
print(classification_report(Y_test, rounded_pred, target_names=labels))

confusion_matrix = confusion_matrix(Y_test, rounded_pred)

sn.set(font_scale = 1.4)
sn.heatmap(confusion_matrix, annot=True, annot_kws={"size":16})
plt.show()

In [ ]:
#Plot a video della training history
plt.figure(figsize=(20, 10))
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()
plt.figure(figsize=(20, 10))
# summarize history for loss
plt.plot(history.history['binary_accuracy'])
plt.plot(history.history['val_binary_accuracy'])
plt.title('model accuracy')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [ ]:
history = np.load("./best_history_VOL.npy", allow_pickle = True)

In [ ]:
# SALVATAGGIO MODELLO:
model.save('./best_model_VOL.h5')

#Salvo i dati "storici" completi dell'apprendimento
with open('./best_history_VOL.npy', 'wb') as file_pi:
        pickle.dump(history.history, file_pi)